# Capstone Week Three - Creating Toronto Data Set

#### Install pandas and numpy

In [81]:
import pandas as pd
import numpy as np

#### Link URL to wiki page and retreive data

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #Assign Wiki page to URL

In [80]:
data = pd.read_html(url, header = 0)[0] # Read HTML table from Wiki Page
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Identify any Postcodes which have a Borough assigned, but not a Neighbourhood (and vice-versa)

In [54]:
test = data[data.Neighbourhood == 'Not assigned']
test[test.Neighbourhood != test.Borough]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [82]:
test1 = data[data.Borough == 'Not assigned']
test1[test1.Borough != test1.Neighbourhood]

,Postcode,Borough,Neighbourhood


#### Replace Neighbourhood for Queen's Park

In [86]:
# Create a list of indices of the cells where the value of Neighbourhood is "Not assigned"
idx_to_change = data.loc[data['Neighbourhood'] == "Not assigned"].index

# Iterate the list of indicies and set Neighbourhood to the value of Borough at the given index
for i in idx_to_change:
    data.iloc[i, 2] = data.iloc[i, 1] 

data.head(9) # Check that this has been replaced in row 9 for Queen's Park

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park


#### Drop rows where Borough is not assigned. As per the checks before, this will get rid of all "Not assigned" values in both columns

In [87]:
data1 = data[data.Borough != 'Not assigned'] # Drop columns where Borough is not assigned
data1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Apply groupby to produce described dataset

In [88]:
final_data = data1.groupby(['Postcode', 'Borough'], as_index = False, sort = False).agg(','.join)
final_data.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
